In [8]:
!pip install numpy
!pip install matplotlib
!pip install opencv-python
!pip install opencv-contrib-python

   ---------------------------------------- 0.0/45.5 MB ? eta -:--:--
   ---- ----------------------------------- 5.5/45.5 MB 25.8 MB/s eta 0:00:02
   --------------- ------------------------ 17.3/45.5 MB 40.4 MB/s eta 0:00:01
   ----------------------- ---------------- 26.5/45.5 MB 43.0 MB/s eta 0:00:01
   -------------------------- ------------- 30.4/45.5 MB 42.0 MB/s eta 0:00:01
   ---------------------------- ----------- 32.5/45.5 MB 34.4 MB/s eta 0:00:01
   ---------------------------- ----------- 32.5/45.5 MB 34.4 MB/s eta 0:00:01
   -------------------------------------- - 44.0/45.5 MB 30.1 MB/s eta 0:00:01
   -------------------------------------- - 44.0/45.5 MB 30.1 MB/s eta 0:00:01
   -------------------------------------- - 44.0/45.5 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------  44.6/45.5 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------  45.4/45.5 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 45.5/45.5 MB 19.4 

In [19]:
import cv2

def drawBoundingBox(img, bbox):
    # Extract bounding box coordinates
    x, y, w, h = int(bbox[0]),  int(bbox[1]), int(bbox[2]), int(bbox[3])
    # Draw a rectangle around the object
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Add "Tracking" label near the bounding box
    cv2.putText(img, "Tracking", (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

# save the result to a video file
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("C:\\Users\\niluf\\Desktop\\Inha\\opencv\\DIP Project\\assets\\small_instance.mp4")

# tracker = cv2.TrackerKCF_create()    # loses tracking when object is out of frame and when the object is moving fast
# tracker = cv2.legacy.TrackerBoosting_create() # slower than mosse but more accurate
# tracker = cv2.legacy.TrackerMOSSE_create() # supposed to be faster than CSRT but less accurate
tracker = cv2.legacy.TrackerCSRT_create()    # Better fps when tracking moving objects than boosting
# tracker = cv2.legacy.TrackerMedianFlow_create()  # the bounding box will be larger when the object is moving fast
# tracker = cv2.legacy.TrackerTLD_create() # not accurate at all

success, img = cap.read()

bbox = cv2.selectROI("Select Region of Interest", img, False)
tracker.init(img, bbox)

while True:
    timer = cv2.getTickCount()  
    success, img = cap.read()
    if not success or img is None:
        print("Error: Unable to read a frame from the video source.")
        break
  
    success, bbox = tracker.update(img)  

    if success:
        drawBoundingBox(img, bbox)
    else:
        cv2.putText(img, "Lost", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)

    # Calculate and display FPS
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(img, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Write the frame to the output video file
    out.write(img)
    
    cv2.imshow("Track using different trackers", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    
    elif cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        break

cap.release()
out.release()
cv2.destroyAllWindows()